# Census API Example C
Hard code some variables, read list of geographies in from a file, separate this list into sublists of 50 items, loop through sublists to retrieve data, write output to dataframe and csv. It's easier to read long lists of variables in from a file rather than hardcoding them. You cannot insert more than 50 variables (columns or geographies) into one request, so you need to break longer lists apart, and loop through the sublists to make multiple requests. 

Example uses ZIP Code Business Patterns 2017:
https://www.census.gov/data/developers/data-sets/cbp-nonemp-zbp/zbp-api.2017.html

## Set Variables

In [1]:
import pandas as pd, requests, os, json
from IPython.display import clear_output

In [2]:
#Set variables
year='2017'
dsource='zbp'
cols='ESTAB,EMP,PAYANN'
geofile='nyc_zips.csv'
outputcsv='nyc_zbpdata_2017.csv'

In [3]:
base_url = f'https://api.census.gov/data/{year}/{dsource}'
base_url

'https://api.census.gov/data/2017/zbp'

In [4]:
#Read in file of ZIP codes
nyczips=pd.read_csv(geofile, sep=',', dtype={'zipcodes':str})
nyczips.head()

,zipcodes
0,10001
1,10118
2,10120
3,10122
4,10123


In [5]:
#Number of rows and columns
nyczips.shape

(317, 1)

In [6]:
# For item i in a range that is a length of l,
# create an index range for l of n items
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [7]:
#Creates a list of lists, with equal number of items in each sublist
ziplist=list(chunks(nyczips.zipcodes.tolist(),50))
print('Number of chunks:',len(ziplist))

Number of chunks: 7


In [8]:
#Shows items in the first two sublists
print(ziplist[0:2])

[['10001', '10118', '10120', '10122', '10123', '10125', '10130', '10002', '10003', '10114', '10276', '10004', '10041', '10045', '10274', '10275', '10005', '10043', '10081', '10203', '10257', '10260', '10265', '10268', '10269', '10270', '10286', '10006', '10256', '10007', '10008', '10242', '10249', '10009', '10010', '10159', '10160', '10011', '10113', '10161', '10012', '10013', '10213', '10014', '10016', '10138', '10156', '10157', '10158', '10017'], ['10087', '10163', '10164', '10166', '10175', '10178', '10179', '10259', '10261', '10018', '10129', '10019', '10101', '10102', '10104', '10105', '10106', '10107', '10020', '10021', '10131', '10022', '10055', '10126', '10150', '10151', '10155', '10023', '10133', '10024', '10132', '10025', '10026', '10027', '10116', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10108', '10109', '10176', '10185', '10037', '10038']]


## Retrieve Data

In [9]:
#Iterate through index and value of each sublist to retrieve data
emp_data=[]
for i, v in enumerate (ziplist):
    batchzips=','.join(v) #Convert items in sublist to a string separated by commas
    data_url = f'{base_url}?get={cols}&for=zipcode:{batchzips}'
    response=requests.get(data_url)
    if response.status_code==200: #Code 200 = success
        clear_output(wait=True)
        data=response.json()
        if i == 0: #If this is the 1st sublist, we want to append everything    
            for record in data:
                emp_data.append(record)
        else: #If it's not, we don't want to append the column headers again
            for record in data[1:]:
                emp_data.append(record) 
        print('Retrieved data for chunk',i)
    else:
        print('***Problem with retrieval***, response code',response.status_code)
        break
print('Done')

Retrieved data for chunk 6
Done


In [10]:
#Number of ZIP Codes will be different, as not all ZIPs have data
len(emp_data)

267

In [11]:
#Preview nested list created from loop
emp_data[0:5]

[['ESTAB', 'EMP', 'PAYANN', 'zip code'],
 ['336', '6587', '775684', '10118'],
 ['69', '1300', '121640', '10120'],
 ['172', '2021', '148853', '10122'],
 ['177', '1949', '193696', '10123']]

## Generate Output

In [12]:
#Create dataframe from list
empdf=pd.DataFrame(emp_data[1:], columns=emp_data[0]).rename(columns={'zip code':'zipcode'}).set_index('zipcode')
for field in empdf.columns:
    empdf=empdf.astype(dtype={field:'int64'})
empdf.sort_values('EMP',ascending=False, inplace=True)
empdf.head()

,ESTAB,EMP,PAYANN
zipcode,,,
10036,6102,178598,23348581
10001,7277,153375,10437332
10019,4782,151434,23024243
10017,4979,148113,22369544
10022,6239,141761,24364428


In [13]:
#Write data out to csv file
empdf.to_csv(outputcsv)